In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path, shuffle=False)
    problem_files = np.array(data['filenames'])
    problem_answers = np_utils.to_categorical(np.array(data['target']), 47)
    return problem_files, problem_answers

# load train, test, and validation datasets
train_files, train_targets = load_dataset('./data/train')
valid_files, valid_targets = load_dataset('./data/validate')
test_files, test_targets = load_dataset('./data/test')

# load list of math answers
math_answers = [item[13:-1] for item in glob("./data/train/*/")]

# print statistics about the dataset
print('There are %d total answer categories.' % len(math_answers))
print('There are %s total math images.' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training math images.' % len(train_files))
print('There are %d validation math images.' % len(valid_files))
print('There are %d test math images.'% len(test_files))

Using TensorFlow backend.


There are 47 total answer categories.
There are 5000 total math images.
There are 3000 training math images.
There are 1000 validation math images.
There are 1000 test math images.


In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(128, 128))
    # convert PIL.Image.Image type to 3D tensor with shape (128, 128, 1)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 128, 128, 1) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1821.52it/s]


In [4]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

### TODO: Define your architecture.
model.add(Conv2D(filters=8, kernel_size=2, padding='same', activation='relu', input_shape=train_tensors.shape[1:]))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=16, kernel_size=2,padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(47, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 8)       104       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 16)        528       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        2080      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
__________

In [5]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 20

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

Train on 3000 samples, validate on 1000 samples
Epoch 1/10
 - 6s - loss: 2.9525 - acc: 0.1733 - val_loss: 2.3098 - val_acc: 0.2620

Epoch 00001: val_loss improved from inf to 2.30976, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/10
 - 3s - loss: 1.8501 - acc: 0.3977 - val_loss: 1.3184 - val_acc: 0.5700

Epoch 00002: val_loss improved from 2.30976 to 1.31842, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 3/10
 - 3s - loss: 0.9732 - acc: 0.6790 - val_loss: 0.4666 - val_acc: 0.8830

Epoch 00003: val_loss improved from 1.31842 to 0.46665, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 4/10
 - 3s - loss: 0.4287 - acc: 0.8663 - val_loss: 0.1272 - val_acc: 0.9720

Epoch 00004: val_loss improved from 0.46665 to 0.12724, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 5/10
 - 3s - loss: 0.1949 - acc: 0.9407 - val_loss: 0.0425 - val_acc: 0.9930

Epoch 00005: val_loss improved from 0.12724 to 0.04253, saving model to 

In [7]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [8]:
# get index of predicted dog breed for each image in test set
math_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(math_predictions)==np.argmax(test_targets, axis=1))/len(math_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 100.0000%


In [9]:
categories = list(set(list(map(int, math_answers))))

In [10]:
categories[np.argmax(model.predict(path_to_tensor('./human/test.png')))]

3

In [11]:
np.argmax(model.predict(path_to_tensor('./human/test.png')))

3